In [74]:
import re
from collections import Counter

def words(text): return re.findall("[a-zA-z']+", text.replace("_", " ").replace("[", " ").replace("]", " ").lower())
# def words(text): return re.findall(r'\w+', text.replace("_", "").lower())
all_words = words(open('big.txt').read())
WORDS = Counter(all_words)

letters = "abcdefghijklmnopqrstuvwxyz"

tmp_BIGRAMS_word = {}
for i in xrange(len(all_words) - 1):
    if all_words[i] not in tmp_BIGRAMS_word:
        tmp_BIGRAMS_word[all_words[i], all_words[i+1]] = 0
    tmp_BIGRAMS_word[all_words[i], all_words[i+1]] = tmp_BIGRAMS_word[all_words[i], all_words[i+1]] + 1

BIGRAMS_word = {}
for qey in tmp_BIGRAMS_word.keys():
    if qey[0] not in BIGRAMS_word:
        BIGRAMS_word[qey[0]] = {}
    BIGRAMS_word[qey[0]][qey[1]] = tmp_BIGRAMS_word[qey]
for qey in BIGRAMS_word.keys():
    
    for qeey in BIGRAMS_word[qey].keys():
        BIGRAMS_word[qey][qeey] = BIGRAMS_word[qey][qeey] + 1.0
    for qeey in BIGRAMS_word[qey].keys():
        tmp_sum = sum(BIGRAMS_word[qey].values())
        BIGRAMS_word[qey][qeey] = (BIGRAMS_word[qey][qeey] + 0.0) / tmp_sum    
    
tmp_TRIGRAMS_letter = {}
for all_word in all_words:
    all_word = " " + all_word + " "
    for i in xrange(len(all_word) - 2):
        if (all_word[i], all_word[i + 1], all_word[i + 2]) not in tmp_TRIGRAMS_letter:
            tmp_TRIGRAMS_letter[(all_word[i], all_word[i + 1], all_word[i + 2])] = 0
        tmp_TRIGRAMS_letter[(all_word[i], all_word[i + 1], all_word[i + 2])] = \
        tmp_TRIGRAMS_letter[(all_word[i], all_word[i + 1], all_word[i + 2])] + 1
    
TRIGRAMS_letter = {}
for qey_1 in letters+" ":
    for qey_2 in letters+" ":
        for qey_3 in letters+" ":
            if (qey_1, qey_2, qey_3) not in TRIGRAMS_letter:        
                TRIGRAMS_letter[(qey_1, qey_2, qey_3)] = tmp_TRIGRAMS_letter.get((qey_1, qey_2, qey_3), 0.0) + 1.0
for qey_1 in letters+" ":
    for qey_2 in letters+" ":
        tmp_sum = 0
        for qey_3 in letters+" ":
            tmp_sum = tmp_sum + TRIGRAMS_letter[(qey_1, qey_2, qey_3)]
        for qey_3 in letters+" ":
            TRIGRAMS_letter[(qey_1, qey_2, qey_3)] = TRIGRAMS_letter[(qey_1, qey_2, qey_3)] / tmp_sum

In [75]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return (WORDS[word] + 0.0) / N

In [76]:
prob_str = 'jx cojwk xr mwey gnub, lw rwwk mx swwz xnh chour exkw-urmwpputwrew rx qommwh lyom jumnomuxr uj. Omywhlujw, jxqwcxkv ojsj nj mx kx wraurwwhura lxhs,, mywr vxn qov jov "lwpp,, hwewrmpv I yoiwrm lhummwr orv exkwj.." lyom o ywpp...'
prob_words = words(prob_str)

In [77]:
import numpy as np
from random import random
from random import randint
from random import sample
from math import log
from math import exp

In [78]:
num_iter = 10

max_prob = None
actual_iter = 0

for this_iter in xrange(num_iter):
    
    cur_str = " ".join(prob_words)
    tmp_str = " " + cur_str + " "
    prev_str = cur_str

    cur_prob = 0
    for i in xrange(len(tmp_str) - 2):
        cur_prob = cur_prob + log(TRIGRAMS_letter[tmp_str[i], tmp_str[i+1], tmp_str[i+2]])
    tmp_str_tmp = cur_str.split(" ")
    for tmp_word in tmp_str_tmp:
        cur_prob = cur_prob + max(P(tmp_word), log(0.0000001))
    for i in xrange(len(tmp_str_tmp) - 1):
        cur_prob = cur_prob + log(BIGRAMS_word.get(tmp_str_tmp[i], {}).get(tmp_str_tmp[i + 1], 0.0000001))
    
    if max_prob == None:
        max_prob = cur_prob
    
    counta = 0
    while counta < 10000:
        randint_a, randint_b = sample(xrange(len(letters)), 2)
        swap_char, can_char = letters[randint_a], letters[randint_b]

        tmp_prob = 0
        tmp_str = " " + cur_str.replace(swap_char, "*").replace(can_char, swap_char).replace("*", can_char)  + " "    

        for i in xrange(len(tmp_str) - 2):
            tmp_prob = tmp_prob + log(TRIGRAMS_letter[tmp_str[i], tmp_str[i+1], tmp_str[i+2]]) 

        tmp_str_tmp = tmp_str.strip().split(" ")
        for tmp_word in tmp_str_tmp:
            tmp_prob = tmp_prob + max(P(tmp_word), log(0.0000001))
        for i in xrange(len(tmp_str_tmp) - 1):
            tmp_prob = tmp_prob + log(BIGRAMS_word.get(tmp_str_tmp[i], {}).get(tmp_str_tmp[i + 1], 0.0000001))

        this_random = random()
        this_not_random = exp(tmp_prob-cur_prob)
        if this_random < this_not_random:
            cur_prob = tmp_prob
            cur_str = cur_str.replace(swap_char, "*").replace(can_char, swap_char).replace("*", can_char)
            prev_str = cur_str

            actual_iter = actual_iter + 1

            if cur_prob > max_prob:
                counta = 0
                max_prob = cur_prob
                max_str = prev_str

                if actual_iter % 1 == 0:
                    print "This iter = " + str(actual_iter) + " | Swap_char = " + swap_char + " | Can_char = " + can_char + \
                    " | Random = " + str(this_random) + " | Not Random = " + str(this_not_random) + \
                    " | Cur prob = " + str(max_prob) + " | Cur str = " + max_str
            else:
                counta = counta + 1
        else:
            counta = counta + 1

This iter = 1 | Swap_char = t | Can_char = b | Random = 0.433478327126 | Not Random = 3319.5242755 | Cur prob = -1636.49155132 | Cur str = jx cojwk xr mwey gnut lw rwwk mx swwz xnh chour exkw urmwppubwrew rx qommwh lyom jumnomuxr uj omywhlujw jxqwcxkv ojsj nj mx kx wraurwwhura lxhs mywr vxn qov jov lwpp hwewrmpv i yoiwrm lhummwr orv exkwj lyom o ywpp
This iter = 2 | Swap_char = v | Can_char = b | Random = 0.184112160563 | Not Random = 5.55297805496 | Cur prob = -1634.77721695 | Cur str = jx cojwk xr mwey gnut lw rwwk mx swwz xnh chour exkw urmwppuvwrew rx qommwh lyom jumnomuxr uj omywhlujw jxqwcxkb ojsj nj mx kx wraurwwhura lxhs mywr bxn qob job lwpp hwewrmpb i yoiwrm lhummwr orb exkwj lyom o ywpp
This iter = 3 | Swap_char = l | Can_char = e | Random = 0.128365789993 | Not Random = 18400712.2122 | Cur prob = -1618.04931703 | Cur str = jx cojwk xr mwly gnut ew rwwk mx swwz xnh chour lxkw urmwppuvwrlw rx qommwh eyom jumnomuxr uj omywheujw jxqwcxkb ojsj nj mx kx wraurwwhura exhs mywr bxn 

This iter = 79 | Swap_char = w | Can_char = h | Random = 0.716702279599 | Not Random = 3.74387380405 | Cur prob = -1331.2590171 | Cur str = pa zopir al nivh bdum si liir na kiiw adc zcoul vari ulniffutilvi la gonnic shon pundonual up onhicsupi pagizare opkp dp na ra ilyuliiculy sack nhil ead goe poe siff civilnfe x hoxiln scunnil ole varip shon o hiff
This iter = 82 | Swap_char = q | Can_char = j | Random = 0.126524217975 | Not Random = 5.27586206897 | Cur prob = -1331.16057703 | Cur str = pa zopir al nivh bdum si liir na kiij adc zcoul vari ulniffutilvi la gonnic shon pundonual up onhicsupi pagizare opkp dp na ra ilyuliiculy sack nhil ead goe poe siff civilnfe x hoxiln scunnil ole varip shon o hiff
This iter = 84 | Swap_char = b | Can_char = w | Random = 0.2261659949 | Not Random = 33.4925440368 | Cur prob = -1328.26594405 | Cur str = pa zopir al nivh xdum si liir na kiij adc zcoul vari ulniffutilvi la gonnic shon pundonual up onhicsupi pagizare opkp dp na ra ilyuliiculy sack nhil ead

This iter = 256 | Swap_char = j | Can_char = v | Random = 0.563893849335 | Not Random = 19.7808082072 | Cur prob = -956.473656936 | Cur str = so gased on tech quiv we need to keep our grain code intellizence no matter what situation is atherwise somegody asks us to do enfineerinf work then you may say well recently x haxent written any codes what a hell
This iter = 257 | Swap_char = z | Can_char = v | Random = 0.063610765835 | Not Random = 831679.312124 | Cur prob = -942.842454733 | Cur str = so gased on tech quiz we need to keep our grain code intellivence no matter what situation is atherwise somegody asks us to do enfineerinf work then you may say well recently x haxent written any codes what a hell
This iter = 259 | Swap_char = x | Can_char = f | Random = 0.755716885332 | Not Random = 106.892717506 | Cur prob = -939.068683427 | Cur str = so gased on tech quiz we need to keep our grain code intellifence no matter what situation is atherwise somegody asks us to do enxineerinx work th

In [79]:
mapping = {}
tmp_str = " ".join(prob_words)
for i in xrange(len(max_str)):
    if tmp_str[i] in letters:
        mapping[tmp_str[i]] = max_str[i]

In [82]:
final_str = prob_str
print ''.join([mapping.get(char.lower(), char) for char in final_str])

so based on tech quiz, we need to keep our brain code-intellifence no matter what situation is. atherwise, somebody asks us to do engineering work,, then you may say "well,, recently v havent written any codes.." what a hell...
